In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns

In [144]:
from datetime import datetime, timedelta

### 마이비 카드 데이터

In [6]:
%%time
mybicard = pd.read_parquet('/home/seho/Passenger_Demand/data/mybicard.parquet', engine='pyarrow')

CPU times: user 22.4 s, sys: 8.42 s, total: 30.8 s
Wall time: 14.3 s


In [7]:
%%time
mybicard = mybicard.drop_duplicates()

CPU times: user 43.4 s, sys: 5.05 s, total: 48.4 s
Wall time: 48.3 s


In [8]:
# 수집일자 데이트 포맷으로 변환
mybicard["collectdate"] = pd.to_datetime(mybicard["collectdate"], format = "%Y%m%d")

In [9]:
# 전송일자 데이트 포맷으로 변환
mybicard["transdate"] = pd.to_datetime(mybicard["transdate"], format = "%Y%m%d %H:%M:%S")

In [10]:
# 요일, 날짜, 월 변수 생성
mybicard["dayofweek"] = mybicard["transdate"].dt.dayofweek
dow_dict = {0:"월", 1:"화", 2:"수", 3:"목", 4:"금", 5:"토", 6:"일"}
mybicard["dayofweek"] = mybicard["dayofweek"].replace(dow_dict)
mybicard["day"] = mybicard["transdate"].dt.day
mybicard["month"] = mybicard["transdate"].dt.month
mybicard["hour"] = mybicard["transdate"].dt.hour

In [11]:
# 전체 승객 수 변수 생성(일반 + 학생 + 아동)
mybicard["totalcnt"] = mybicard[["normalcnt", "studentcnt", "childcnt"]].sum(axis = 1)

In [12]:
# route_nm에 공백이 포함되어 있어 공백 제거
mybicard["route_nm"] = mybicard["route_nm"].replace("\s", "", regex = True)

In [13]:
mybicard = mybicard.sort_values(["transdate", "seq"]).reset_index(drop=True)

In [14]:
mybicard = mybicard.rename(columns = {"stop_id" : "mybi_stop_id"})

### 정류장 X,Y 좌표 추가

In [32]:
# 경주시, 양산시, 울산광역시, 부산광역시
bus_stop = pd.read_csv("/home/seho/Passenger_Demand/data/울산광역시_버스 정류소 위치 정보_20200531.csv", encoding = "euc-kr")
bus_stop = bus_stop.loc[bus_stop["권역"] == "울산광역시"]
bus_stop.columns = ["stop_nm", "stop_id", "longitude", "latitude", "city"]

In [33]:
bus_stop_401_1 = pd.read_csv("/home/seho/Passenger_Demand/data/401_율리_꽃바위.csv", encoding = "euc_kr")
bus_stop_401_2 = pd.read_csv("/home/seho/Passenger_Demand/data/401_꽃바위_율리.csv", encoding = "euc_kr")
bus_stop_401 = pd.concat([bus_stop_401_1, bus_stop_401_2])
bus_stop_401.columns = ["mybi_stop_id", "stop_id"]

In [34]:
# 마이비 정류장 ID -> 공공 데이터 정류쟝 ID로 매핑
mybicard_401 = pd.merge(mybicard_401, bus_stop_401, on = "mybi_stop_id", how = "inner")

In [35]:
mybicard_401 = pd.merge(mybicard_401, bus_stop[["stop_id", "longitude", "latitude"]], on = "stop_id")

### 날씨 데이터 추가

In [382]:
weather_data = pd.read_parquet("/home/seho/Passenger_Demand/data/weather_2018.parquet")

In [383]:
weather_data = weather_data.loc[:, ["tm", "ta", "hm", "rn", "dc10Tca",  "dsnw", "wd", "ws"]]
weather_data = weather_data.rename(columns = {"tm" : "시간",
                                              "ta" : "기온",
                                              "hm" : "습도",
                                              "rn" : "강수량",
                                              "dc10Tca" : "전운량",
                                              "dsnw" : "적설량",
                                              "wd" : "풍향",
                                              "ws" : "풍속"})
weather_data["시간"] = pd.to_datetime(weather_data["시간"], format = "%Y-%m-%d %H:%M")

In [384]:
for col in weather_data.columns:
    if col == "시간":
        continue
    print(col)
    weather_data[col] = weather_data[col].replace("", "0.0").astype(float)
    weather_data[col] = weather_data[col].astype(float)

기온
습도
강수량
전운량
적설량
풍향
풍속


In [385]:
weather_data["time_hours"] = weather_data["시간"].dt.strftime("%Y-%m-%d %H")

In [386]:
mybicard_401["transdate_hours"] = mybicard_401["transdate"].dt.strftime("%Y-%m-%d %H")

In [387]:
mybicard_401_wt = pd.merge(mybicard_401, weather_data, left_on = "transdate_hours", right_on = "time_hours")
mybicard_401_wt = mybicard_401_wt.drop(["transdate_hours", "time_hours", "시간"], 1)

### 사고 데이터

In [ ]:
accident_data = pd.read_parquet("/home/seho/Passenger_Demand/data/accident_data.parquet")

In [ ]:
accident_data